In [ ]:
import mne
mne.set_log_level("CRITICAL")
import numpy as np
import double_dipper
from double_dipper import dataset
from double_dipper import constants
from double_dipper.io import filePairs, partition
from double_dipper.dataset import mapDataset, subset
from double_dipper.ml import cross_validation, temporal_cross_validation

In [ ]:
def labeller(meta):
    strat = meta["strategy"]
    if strat is None: return None
    if strat.lower().startswith("fact"):        return 0
    elif strat.lower().startswith("procedure"): return 1
    else:                                       return None

In [ ]:
divider = lambda meta: meta["id"]

In [ ]:
subjNos = [i for i in range(1,11+1) if i != 5]
pairs = filePairs(*[f"cleaned/main/{i}" for i in subjNos])
dataset = partition(divider, labeller, pairs)

In [ ]:
sfreq = double_dipper.constants.sfreq
mapDataset(lambda data: mne.filter.filter_data(data, sfreq, l_freq=1, h_freq=32), dataset)
mapDataset(lambda data: data.reshape([data.shape[0], np.prod(data.shape[1:])]), dataset)

In [ ]:
(precision, recall) = cross_validation(dataset)
f1 = 2 * (precision * recall) / (precision + recall)
print(f1)

## Temporal Cross Validation

In [18]:
divider = lambda meta: (meta["id"], meta["epoch"])
subjNos = [i for i in range(1,11+1) if i != 5]
pairs = filePairs(*[f"cleaned/main/{i}" for i in subjNos])
dset = partition(divider, labeller, pairs)

#### Intra-Subject

In [27]:
subjNo = 11
subjSet = dataset.subset(lambda k: k[0] == subjNo, dset)
keys = sorted(subjSet.keys(), key= lambda k: k[1])
X = np.concatenate([subjSet[k]["x"] for k in keys], axis = 0)
Y = np.concatenate([subjSet[k]["y"] for k in keys], axis = 0)

In [53]:
Y.shape

(112,)

In [28]:
channel_inds = [i for (i, ch) in enumerate(constants.channel_names) if ch.startswith("Fp")]
X = X[:, channel_inds, :]
X = mne.filter.filter_data(X, constants.sfreq, l_freq=1, h_freq=32)
X = X.reshape([X.shape[0], np.prod(X.shape[1:])])

In [29]:
from tensorflow import keras
np.random.seed(0)

In [37]:
split_ind = int(len(X) * .7)
trainX = X[:split_ind]
trainY = Y[:split_ind]
testX = X[split_ind:]
testY = Y[split_ind:]

In [59]:
inbalance = len(trainY) // 2 - sum(trainY)
inds = np.random.choice(np.argwhere(trainY == 1).flatten(), size=inbalance)

In [60]:
resampledX = np.concatenate( [trainX, trainX[inds]], axis = 0)
resampledY = np.concatenate( [trainY, trainY[inds]], axis = 0)

In [61]:
model = keras.Sequential([
    keras.layers.Dense(256, input_shape=(X.shape[-1],), activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="adam",loss="binary_crossentropy")
model.fit(resampledX, resampledY, steps_per_epoch=1, epochs=200, batch_size=len(trainX))

Epoch 1/200
1/1 [==============================] - 0s 376ms/step - loss: 0.6931
Epoch 2/200
1/1 [==============================] - 0s 34ms/step - loss: 0.6917
Epoch 3/200
1/1 [==============================] - 0s 27ms/step - loss: 0.6899
Epoch 4/200
1/1 [==============================] - 0s 30ms/step - loss: 0.6880
Epoch 5/200
1/1 [==============================] - 0s 29ms/step - loss: 0.6859
Epoch 6/200
1/1 [==============================] - 0s 26ms/step - loss: 0.6836
Epoch 7/200
1/1 [==============================] - 0s 33ms/step - loss: 0.6811
Epoch 8/200
1/1 [==============================] - 0s 35ms/step - loss: 0.6785
Epoch 9/200
1/1 [==============================] - 0s 35ms/step - loss: 0.6758
Epoch 10/200
1/1 [==============================] - 0s 44ms/step - loss: 0.6730
Epoch 11/200
1/1 [==============================] - 0s 31ms/step - loss: 0.6700
Epoch 12/200
1/1 [==============================] - 0s 51ms/step - loss: 0.6670
Epoch 13/200
1/1 [==============================

Epoch 104/200
1/1 [==============================] - 0s 35ms/step - loss: 0.0381
Epoch 105/200
1/1 [==============================] - 0s 46ms/step - loss: 0.0362
Epoch 106/200
1/1 [==============================] - 0s 40ms/step - loss: 0.0345
Epoch 107/200
1/1 [==============================] - 0s 46ms/step - loss: 0.0328
Epoch 108/200
1/1 [==============================] - 0s 40ms/step - loss: 0.0313
Epoch 109/200
1/1 [==============================] - 0s 36ms/step - loss: 0.0298
Epoch 110/200
1/1 [==============================] - 0s 29ms/step - loss: 0.0284
Epoch 111/200
1/1 [==============================] - 0s 26ms/step - loss: 0.0271
Epoch 112/200
1/1 [==============================] - 0s 26ms/step - loss: 0.0259
Epoch 113/200
1/1 [==============================] - 0s 24ms/step - loss: 0.0248
Epoch 114/200
1/1 [==============================] - 0s 26ms/step - loss: 0.0237
Epoch 115/200
1/1 [==============================] - 0s 36ms/step - loss: 0.0227
Epoch 116/200
1/1 [=========

In [62]:
preds = model.predict(testX)

In [63]:
preds

array([[5.71370125e-04],
       [1.69409215e-02],
       [1.17639989e-01],
       [1.58669531e-01],
       [1.40219331e-02],
       [2.17836797e-02],
       [3.91375422e-01],
       [1.45290494e-02],
       [4.71561790e-01],
       [6.15015447e-01],
       [5.05611300e-03],
       [9.31193233e-01],
       [4.80238199e-02],
       [4.28140163e-04],
       [6.81021810e-03],
       [2.57041216e-01],
       [3.65032554e-02],
       [4.72668409e-02],
       [2.31552660e-01],
       [2.40221620e-03],
       [9.19729948e-01],
       [5.68271279e-02],
       [1.70634329e-01],
       [8.51985812e-03],
       [2.83661485e-02],
       [6.62951469e-02],
       [2.94607818e-01],
       [1.12809390e-01],
       [4.07761335e-03],
       [1.09571695e-01],
       [2.14278698e-04],
       [1.20568484e-01],
       [8.52369331e-03],
       [2.07939684e-01]], dtype=float32)

In [64]:
sum(trainY)

18

In [65]:
len(trainY)

78

In [ ]:
i